In [3]:
# Load dependencies
import csv
import pandas as pd
import numpy as np
import os
import gc
import warnings
import itertools as it
import DataProcessFunctions as DP
import PredictionStep1 as pred
import SupportFunctions as supp
import LinearModelEstimation as lm
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import statsmodels.api as sm  
import statsmodels.formula.api as smf
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression as lr
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras import callbacks
from matplotlib.pyplot import cm
import time as time 


%load_ext autoreload
%autoreload 2

# Disable scientific notation (e) in numpy
# Disable futurewarnings
np.set_printoptions(suppress=True)
warnings.simplefilter(action='ignore', category=FutureWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Initialize warning log container 
log = list()

In [4]:
# Load Pre-processed data from Data Processing PCA.ipynb
FM_data = pd.read_csv(os.path.dirname(os.getcwd()) + '\\FM2_data.csv')
returns = pd.read_csv(os.path.dirname(os.getcwd()) + '\\returns2_data.csv').set_index(["permno", "date"])
industry_code = pd.read_csv(os.path.dirname(os.getcwd()) + '\\industry2_codes.csv').set_index(["permno", "date"])

# Load excess SP500 log returns data 
SP500 = pd.read_excel(os.path.dirname(os.getcwd()) + '\\SP500.xlsx')["Cum return"]

supp.downcast(FM_data)
supp.downcast(returns)
supp.downcast(industry_code)

Before downcast: 1.769 GB and float64    102
int64        3
dtype: int64
After downcast: 0.878 GB and float32    102
int32        2
int8         1
dtype: int64
Before downcast: 0.026 GB and float64    1
dtype: int64
After downcast: 0.018 GB and float32    1
dtype: int64
Before downcast: 0.026 GB and float64    1
dtype: int64
After downcast: 0.018 GB and float32    1
dtype: int64


In [9]:
# Set number of iterations
ite = 7 # 30 splits --> iteration = [0;29]

# Initialize arrays to store loss and explained variation
loss = pd.DataFrame(columns = range(ite), index = ["LR", "Lasso", "NN"])
xplained_variation = pd.DataFrame(columns = range(ite), index = ["LR", "Lasso", "NN"])

# Initialize arrays to store annual loss and explained variation
loss_annual = pd.DataFrame(columns = range(ite), index = ["LR", "Lasso", "NN"])
xplained_variation_annual = pd.DataFrame(columns = range(ite), index = ["LR", "Lasso", "NN"])

# Initialize arrays to store predicted and actual returns used in portfolio sorts later
LR_pred_actual = pd.DataFrame()
lasso_pred_actual = pd.DataFrame()
NN_pred_actual = pd.DataFrame()

for i in range(ite):
    
    # Compute training, validation, and test set
    training, validation, test = DP.complete_data_process(industry_code, returns, FM_data, iteration = i)
    
    # Split in X and Y
    training_x, training_y = pred.XY_split(training)
    validation_x, validation_y = pred.XY_split(validation)
    test_x, test_y = pred.XY_split(test)
    
    # ---------------------------
    
    # Algorithm 1: Simple Linear (PCR in Gu, kelly, and Xiu (2020) due to PCA)
    # Fit model on training set & predict on test set
    LR = lr().fit(training_x, training_y)
    LR_pred = LR.predict(test_x)
    
    # Algoritm 1: Compute loss and explained varation, and combine 
    # actual and predicted returns. Append all. 
    LR_loss, LR_explained_var, LR_pred_actual_temp, LR_loss_annual, LR_explained_var_annual = pred.to_append(LR_pred, test_y)
    loss.iloc[0, i] = LR_loss
    loss_annual.iloc[0, i] = LR_loss_annual
    xplained_variation.iloc[0, i] = LR_explained_var
    xplained_variation_annual.iloc[0, i] = LR_explained_var_annual
    LR_pred_actual = LR_pred_actual.append(LR_pred_actual_temp)
    
    # ---------------------------
    
    # ALgorithm 2: LASSO
    # Fit model on training set and select tuning parameter based on validation set
    lambda_grid = pred.lambda_grid(training_x, training_y)
    loss_validation = []

    for lamb in lambda_grid:
        lasso = Lasso(alpha = lamb, tol = 0.001).fit(training_x, training_y)
        lasso_pred = lasso.predict(validation_x)
        loss_validation.append(pred.loss_function(lasso_pred, validation_y))

    # Fit model with error minimizing tuning parameter
    lambda_min = lambda_grid[loss_validation.index(min(loss_validation))]
    lasso_min = Lasso(alpha = lambda_min).fit(training_x, training_y)
    lasso_min_pred = lasso_min.predict(test_x)

    # Algorithm 2: Appending 
    lasso_loss, lasso_explained_var, lasso_pred_actual_temp, lasso_loss_annual, lasso_explained_var_annual = pred.to_append(lasso_min_pred, test_y)
    loss.iloc[1, i] = lasso_loss
    loss_annual.iloc[1, i] = lasso_loss_annual
    xplained_variation.iloc[1, i] = lasso_explained_var
    xplained_variation_annual.iloc[1, i] = lasso_explained_var_annual
    lasso_pred_actual = lasso_pred_actual.append(lasso_pred_actual_temp)
    
    # ---------------------------
    
    # ALgorithm 3: NN 
    # Build NN architecture (L1 regularization and batch normalization)
    model = pred.NN(training_x)

    # Define callback for early stopping
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

    # Fit model
    model.fit(training_x, training_y, epochs = 50, batch_size = 500, verbose = 0, validation_data = (validation_x, validation_y), callbacks = [callback])

    # Compute predictions
    NN_pred = model.predict(test_x)
    
    # Algorithm 3: Appending
    NN_loss, NN_explained_var, NN_pred_actual_temp, NN_loss_annual, NN_explained_var_annual = pred.to_append(NN_pred, test_y)
    loss.iloc[2, i] = NN_loss
    loss_annual.iloc[2, i] = NN_loss_annual
    xplained_variation.iloc[2, i] = NN_explained_var
    xplained_variation_annual.iloc[2, i] = NN_explained_var_annual
    NN_pred_actual = NN_pred_actual.append(NN_pred_actual_temp)
    
    # ---------------------------
    
    # Free memory
    del training
    del validation
    del test
    del training_x
    del training_y
    del validation_x
    del validation_y
    del test_x
    del test_y
    
    gc.collect()

 data_processing_new time: 0.43802690505981445
interaction_new time: 7.435297004381815
pca time: 13.762475029627483
Rows Training set: 406519, Row Validation set: 629972, Rows test set: 63913, Columns: 168, Iteration finished: 0, Time: 21.7574209690094, Training Dtypes: float32    102
uint8       66
dtype: int64, Validation Dtypes: float32    102
uint8       66
dtype: int64, Test Dtypes: float32    102
uint8       66
dtype: int64,
1998/1998 [==============================] - 4s 2ms/step
 data_processing_new time: 0.26538562774658203
interaction_new time: 7.35698523124059
pca time: 16.18630340496699
Rows Training set: 444575, Row Validation set: 644834, Rows test set: 63116, Columns: 166, Iteration finished: 1, Time: 23.919417476654054, Training Dtypes: float32    99
uint8      67
dtype: int64, Validation Dtypes: float32    99
uint8      67
dtype: int64, Test Dtypes: float32    99
uint8      67
dtype: int64,


KeyboardInterrupt: 

In [730]:
# Return for each decile at all points in time
LR_deciles_ret = pred.portfolio_sorts_1(LR_pred_actual)
lasso_deciles_ret = pred.portfolio_sorts_1(lasso_pred_actual)
NN_deciles_ret = pred.portfolio_sorts_1(NN_pred_actual)

# Return and prediction for 10-1 portfolios at all points in time
LR_10_1_ret = pred.decile_10_1(LR_deciles_ret)
lasso_10_1_ret = pred.decile_10_1(lasso_deciles_ret)
NN_10_1_ret = pred.decile_10_1(NN_deciles_ret)

# -------------------------------------------------------------------

# Log returns of decile portfolios (cannot accumulate returns in pct.)
LR_deciles_log_ret = LR_deciles_ret.copy()
LR_deciles_log_ret.ret = np.log(1 + LR_deciles_log_ret.ret)
LR_deciles_log_ret.rename(columns = {"ret":"log_ret"}, inplace = True)

lasso_deciles_log_ret = lasso_deciles_ret.copy()
lasso_deciles_log_ret.ret = np.log(1 + lasso_deciles_log_ret.ret)
lasso_deciles_log_ret.rename(columns = {"ret":"log_ret"}, inplace = True)

NN_deciles_log_ret = NN_deciles_ret.copy()
NN_deciles_log_ret.ret = np.log(1 + NN_deciles_log_ret.ret)
NN_deciles_log_ret.rename(columns = {"ret":"log_ret"}, inplace = True)

# -------------------------------------------------------------------

# Log returns of 10-1 portfolio
LR_10_1_log_ret = LR_10_1_ret.copy()
LR_10_1_log_ret.ret = np.log(1 + LR_10_1_log_ret.ret)
LR_10_1_log_ret.rename(columns = {"ret":"log_ret"}, inplace = True)

lasso_10_1_log_ret = lasso_10_1_ret.copy()
lasso_10_1_log_ret.ret = np.log(1 + lasso_10_1_log_ret.ret)
lasso_10_1_log_ret.rename(columns = {"ret":"log_ret"}, inplace = True)

NN_10_1_log_ret = NN_10_1_ret.copy()
NN_10_1_log_ret.ret = np.log(1 + NN_10_1_log_ret.ret)
NN_10_1_log_ret.rename(columns = {"ret":"log_ret"}, inplace = True)

# -------------------------------------------------------------------

# Accumualtive log returns and predicted returns 
# of all deciles (cannot accumulate returns in pct.)
LR_cum_log_ret = pred.portfolio_sorts_acc_return(LR_deciles_log_ret)
lasso_cum_log_ret = pred.portfolio_sorts_acc_return(lasso_deciles_log_ret)
NN_cum_log_ret = pred.portfolio_sorts_acc_return(NN_deciles_log_ret)

# Monthly average return and std. deviation, and annualized SR
# of both predicted and actual returns
LR_mean, LR_std, LR_sr = pred.portfolio_sorts_SR(LR_deciles_ret)
lasso_mean, lasso_std, lasso_sr = pred.portfolio_sorts_SR(lasso_deciles_ret)
NN_mean, NN_std, NN_sr = pred.portfolio_sorts_SR(NN_deciles_ret)

# Monthly average return and std. deviation, and annualized SR 
# for 10-1 portfolio
LR_10_1_mean, LR_10_1_sd, LR_10_1_sr = pred.portfolio_sorts_SR(LR_10_1_ret)
lasso_10_1_mean, lasso_10_1_sd, lasso_10_1_sr = pred.portfolio_sorts_SR(lasso_10_1_ret)
NN_10_1_mean, NN_10_1_sd, NN_10_1_sr = pred.portfolio_sorts_SR(NN_10_1_ret)

In [910]:
# Figure: Cumulative return of all deciles for each ML model
pred.cumulative_ret_fig(data = LR_deciles_log_ret, name = "LR_cumulative_log_ret", market = SP500, save_fig = True, hide = True)
pred.cumulative_ret_fig(data = lasso_deciles_log_ret, name = "lasso_cumulative_log_ret", market = SP500, save_fig = True, hide = True)
pred.cumulative_ret_fig(data = NN_deciles_log_ret, name = "NN_cumulative_log_ret", market = SP500, save_fig = True, hide = True)

In [920]:
# Figure: Cumulative return of 10-1 portfolio for each ML model
pred.portfolio_10_1_fig(name = "10-1_portfolio_cumulative_log_ret", market = SP500, save_fig = True, hide = True, data1 = LR_10_1_log_ret, data2 = lasso_10_1_log_ret, data3 = NN_10_1_log_ret)

In [917]:
# Figure: Cumulative return of 1st and 10th decile of specified ML models 
pred.deciles_10_1_fig(name = "deciles_10_1_cumulative_log_ret", market = SP500, save_fig = True, hide = True, data1 = LR_deciles_log_ret, data2 = lasso_deciles_log_ret, data3 = NN_deciles_log_ret)

In [874]:
# Prep data for export to R so as to customize for tables 

# Table 1: monthly loss and explained variation (Multiply with 100 to get pct.)
monthly_pricing_error = loss.mean(axis = 1) * 100
monthly_xplained_variation = xplained_variation.mean(axis = 1) * 100
table1 = pd.concat([monthly_pricing_error, monthly_xplained_variation], axis = 1)
table1.columns = ["Squared Pricing Error", "Explained Variation"]
table1 = table1.transpose()
table1.columns = ["Linear Regression", "Lasso", "Neural Network"]
table1.to_csv(os.path.dirname(os.getcwd()) + '\\table1_data.csv', header = True, index = True)

''' Incorrect as is. Consult Predictionstep1.py, to_append function
# Table 2: Annual loss and explained variation (Multiply with 100 to get pct.)
annual_pricing_error = loss_annual.mean(axis = 1) * 100
annual_xplained_variation = xplained_variation_annual.mean(axis = 1) * 100
table2 = pd.concat([annual_pricing_error, annual_xplained_variation], axis = 1)
table2.columns = ["Squared Pricing Error", "Explained Variation"]
table2 = table2.transpose()
table2.columns = ["Linear Regression", "Lasso", "Neural Network"]
table2.to_csv(os.path.dirname(os.getcwd()) + '\\table2_data.csv', header = True, index = True)
'''

# Table 3: (Multiply with 100 to get pct.)
table3_LR = pd.concat([LR_mean * 100, LR_std.ret * 100, LR_sr.ret], axis = 1).round(2)
table3_LR.columns = ["Avg", "Pred", "Std", "SR"]
table3_lasso = pd.concat([lasso_mean * 100 , lasso_std.ret * 100, lasso_sr.ret], axis = 1).round(2)
table3_lasso.columns = ["Avg", "Pred", "Std", "SR"]
table3_NN = pd.concat([NN_mean * 100, NN_std.ret * 100, NN_sr.ret], axis = 1).round(2)
table3_NN.columns = ["Avg", "Pred", "Std", "SR"]

# Table 3: Add 10-1 portfolio row for each ML model
LR_portfolio_10_1_row = pd.concat((LR_10_1_mean * 100, LR_10_1_sd.ret * 100, LR_10_1_sr.ret), axis = 1)
LR_portfolio_10_1_row.columns = ["Avg", "Pred", "Std", "SR"]
table3_LR = pd.concat((table3_LR, LR_portfolio_10_1_row), axis = 0).round(2)

lasso_portfolio_10_1_row = pd.concat((lasso_10_1_mean * 100, lasso_10_1_sd.ret * 100, lasso_10_1_sr.ret), axis = 1)
lasso_portfolio_10_1_row.columns = ["Avg", "Pred", "Std", "SR"]
table3_lasso = pd.concat((table3_lasso, lasso_portfolio_10_1_row), axis = 0).round(2)

NN_portfolio_10_1_row = pd.concat((NN_10_1_mean * 100, NN_10_1_sd.ret * 100, NN_10_1_sr.ret), axis = 1)
NN_portfolio_10_1_row.columns = ["Avg", "Pred", "Std", "SR"]
table3_NN = pd.concat((table3_NN, NN_portfolio_10_1_row), axis = 0).round(2)

# Table 4: Tabel for appendix figurerne (skal bruge LR_cum, lasso_cum, NN_cum) og så bare kun ret søjlen. Har så tabel der viser end point for figurene (denne tabel kommer i appendix) 
table4 = pd.concat([LR_cum_log_ret.log_ret, lasso_cum_log_ret.log_ret, NN_cum_log_ret.log_ret], axis = 1).round(3)
table4.columns = ["Linear Regression", "Lasso", "Neural Network"]
table4 = table4.transpose()
table4.to_csv(os.path.dirname(os.getcwd()) + '\\table4_data.csv', header = True, index = True) 

In [1153]:
# Load data for CAPM, FF3, and FF5 
FF5 = pd.read_csv(os.path.dirname(os.getcwd()) + '\\F-F_Research_Data_5_Factors_2x3.csv')
FF5.rename(columns = {'Unnamed: 0':'date'}, inplace = True)
FF5 = FF5[(FF5["date"] >= 198701) & (FF5["date"] < 200301)] # 201701
FF5 = FF5.set_index('date')
FF5 = FF5[["Mkt-RF", "SMB", "HML", "RMW", "CMA"]]

# Define FF3 and CAPM from FF5
FF3 = FF5[["Mkt-RF", "SMB", "HML"]]
CAPM = FF5["Mkt-RF"]

# LR regressions
CAPM_LR = pred.linearmodel(LR_10_1_ret["ret"] * 100, CAPM)
FF3_LR = pred.linearmodel(LR_10_1_ret["ret"] * 100, FF3)
FF5_LR = pred.linearmodel(LR_10_1_ret["ret"] * 100, FF5)

# Lasso regressions
CAPM_lasso = pred.linearmodel(lasso_10_1_ret["ret"] * 100, CAPM)
FF3_lasso = pred.linearmodel(lasso_10_1_ret["ret"] * 100, FF3)
FF5_lasso = pred.linearmodel(lasso_10_1_ret["ret"] * 100, FF5)

# NN regressions
CAPM_NN = pred.linearmodel(NN_10_1_ret["ret"] * 100, CAPM)
FF3_NN = pred.linearmodel(NN_10_1_ret["ret"] * 100, FF3)
FF5_NN = pred.linearmodel(NN_10_1_ret["ret"] * 100, FF5)

# CAPM table
CAPM_params, CAPM_tvalues = pred.LM_tables(CAPM_LR, CAPM_lasso, CAPM_NN)
CAPM_params.index = ["Constant", "Mkt-RF"]
CAPM_tvalues.index = ["Constant", "Mkt-RF"]
CAPM_params.T.to_csv(os.path.dirname(os.getcwd()) + '\\CAPM_params.csv', header = True, index = True) 
CAPM_tvalues.T.to_csv(os.path.dirname(os.getcwd()) + '\\CAPM_tvalues.csv', header = True, index = True) 

# FF3 table
FF3_params, FF3_tvalues = pred.LM_tables(FF3_LR, FF3_lasso, FF3_NN)
FF3_params.index = ["Constant", "Mkt-RF", "SMB", "HML"]
FF3_tvalues.index = ["Constant", "Mkt-RF", "SMB", "HML"]
FF3_params.T.to_csv(os.path.dirname(os.getcwd()) + '\\FF3_params.csv', header = True, index = True) 
FF3_tvalues.T.to_csv(os.path.dirname(os.getcwd()) + '\\FF3_tvalues.csv', header = True, index = True) 

# FF5 table 
FF5_params, FF5_tvalues = pred.LM_tables(FF5_LR, FF5_lasso, FF5_NN)
FF5_params.index = ["Constant", "Mkt-RF", "SMB", "HML", "RMW", "CMA"]
FF5_tvalues.index = ["Constant", "Mkt-RF", "SMB", "HML", "RMW", "CMA"]
FF5_params.T.to_csv(os.path.dirname(os.getcwd()) + '\\FF5_params.csv', header = True, index = True) 
FF5_tvalues.T.to_csv(os.path.dirname(os.getcwd()) + '\\FF5_tvalues.csv', header = True, index = True) 

In [1154]:
print(FF3_LR.summary())

                            OLS Regression Results                            
Dep. Variable:                    ret   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     5.636
Date:                Sat, 03 Dec 2022   Prob (F-statistic):            0.00102
Time:                        19:30:58   Log-Likelihood:                -620.64
No. Observations:                 192   AIC:                             1249.
Df Residuals:                     188   BIC:                             1262.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.0186      0.459      4.402      0.0

In [ ]:
# Skal jeg lave en normals OLS på ikke PCA data? bare som totalt benchmark? -- køre det på FM_data + industry codes så -- ingen interaktion terms eller PCA 